In [1]:
## The neuropath data at Emory ADRC is currently not particularly well organized
## and naming schema is inconsistent.  Working on developing a schema 
import os,sys, json, girder_client
import dagSecrets as dgs
from collections import Counter
import pandas as pd
import pickle
%load_ext autoreload
%autoreload 2
import brain_region_maps as brm
import adrcSchemaHelpers as hlprs
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
gc = girder_client.GirderClient(apiUrl=dgs.cbApiUrl)
gc.authenticate(apiKey=dgs.cbApiKey)

{'_id': '6102efcc7d966fafaa736885'}

In [6]:
## Load all of the item data for all WSI images
npPickleFile = 'npDemoData.pickle'
savePickle = True
reload = True

if os.path.isfile(npPickleFile) and not reload:
    with open(npPickleFile,"rb") as fp:
        npSlideDict=  pickle.load(fp)
else:
    print("RELOADING DATA")
    npSlideDict = hlprs.getADRCcollection(gc,rootCollectionName='Emory_ADRC')
    if savePickle:
        with open(npPickleFile,"wb") as fp:
            pickle.dump(npSlideDict,fp)

RELOADING DATA
[{'_accessLevel': 2, '_id': '5f4ad0e0239fccd2dfc30887', '_modelType': 'collection', '_textScore': 11.0, 'created': '2020-08-29T22:04:16.382000+00:00', 'description': 'All WSI from Emory neuropathology core.', 'meta': {}, 'name': 'Emory_ADRC', 'public': False, 'publicFlags': [], 'size': 62919061850181, 'updated': '2020-08-29T22:04:16.382000+00:00'}, {'_accessLevel': 2, '_id': '5f4b1806239fccd2dfc3e27b', '_modelType': 'collection', '_textScore': 0.5294117647058824, 'created': '2020-08-30T03:07:50.966000+00:00', 'description': 'WSIs used in JCs PhD project. Pathologists will annotate the WSIs in this collection. These WSIs are copies of WSIs from the Emory_ADRC collection or collections from other institutions.', 'meta': {}, 'name': 'BraakStudy', 'public': False, 'publicFlags': [], 'size': 7397610871234, 'updated': '2020-08-30T03:07:50.966000+00:00'}]


In [21]:
hlprs.scanMetadata(gc,npSlideDict['2021_E21-30'],updateGirder=True)

({'Processed': 83, 'girderUpdates': 79, 'slidesToUpdate': 79}, [])

In [25]:
for fldr in npSlideDict.keys():
    hlprs.scanMetadata(gc,npSlideDict[fldr],updateGirder=True)

This slidename failed to match and is not a control ADRC36-04_LHp_pTDP_1to10k.ndpi
This slidename failed to match and is not a control ADRC36-04_F_pTDP-1to10K.ndpi
This slidename failed to match and is not a control ADRC36-04_RHp_pTDP_1to10k.ndpi
This slidename failed to match and is not a control E39-44_TAU.ndpi
This slidename failed to match and is not a control E39-44_BLANK.ndpi
This slidename failed to match and is not a control E39-44_AB.ndpi
This slidename failed to match and is not a control OS89-44_Ab.ndpi
This slidename failed to match and is not a control OS89-44_Ab_2.ndpi
This slidename failed to match and is not a control OS89-44_Ab_3.ndpi
This slidename failed to match and is not a control PSTVE_CNTRL_FOR_OS89-44_AB.ndpi
This slidename failed to match and is not a control E39-44_SYN.ndpi
This slidename failed to match and is not a control 1028860.svs
This slidename failed to match and is not a control 1028861.svs
This slidename failed to match and is not a control 1028862.

This slidename failed to match and is not a control 1013369.svs
This slidename failed to match and is not a control E00-19_Am_pTDP-1to10k.ndpi
This slidename failed to match and is not a control E00-19_LHp_pTDP-1to10k.ndpi
This slidename failed to match and is not a control E00-19_RHp_pTDP-1to10K.ndpi
This slidename failed to match and is not a control 1013600.svs
This slidename failed to match and is not a control 0500-04-4-t-asyn-plus.ndpi
This slidename failed to match and is not a control OS00-41_1.ndpi
This slidename failed to match and is not a control ADRC51-2_Hp2_TAU.ndpi
This slidename failed to match and is not a control E01-06_1.ndpi
This slidename failed to match and is not a control E01-06_2.ndpi
This slidename failed to match and is not a control E01-06_7.ndpi
This slidename failed to match and is not a control E01-06_8.ndpi
This slidename failed to match and is not a control E01-06_9.ndpi
This slidename failed to match and is not a control ADRC51-10_HF_tau.ndpi
This slid

This slidename failed to match and is not a control 0S91-60_C_UBIQUITIN.ndpi
This slidename failed to match and is not a control E06-06_HE_1J.ndpi
This slidename failed to match and is not a control E06-06_HE_1L.ndpi
This slidename failed to match and is not a control E06-06_HE_1M.ndpi
This slidename failed to match and is not a control E06-06_HE_1N.ndpi
This slidename failed to match and is not a control E06-06_HE_1O.ndpi
This slidename failed to match and is not a control E06-06_HE_1P.ndpi
This slidename failed to match and is not a control E06-06_HE_1Q.ndpi
This slidename failed to match and is not a control E06-06_HE_1R.ndpi
This slidename failed to match and is not a control E06-06_HE_1S.ndpi
This slidename failed to match and is not a control E06-06_HE_1T.ndpi
This slidename failed to match and is not a control E06-06_HE_1U.ndpi
This slidename failed to match and is not a control E06-06_HE_1W.ndpi
This slidename failed to match and is not a control E06-06_HE_1X.ndpi
This slidenam

This slidename failed to match and is not a control E06-18_AB_US3_1P.ndpi
This slidename failed to match and is not a control ADRC56-41_AB_4G8_US2_2C.ndpi
This slidename failed to match and is not a control E06-18_ASYN_US2_1D.ndpi
This slidename failed to match and is not a control ADRC56-41_aSyn_US3_2C.ndpi
This slidename failed to match and is not a control E06-52_CG_1_Ubiq_minus.ndpi
This slidename failed to match and is not a control E06-52-CG-1-Ubiq.ndpi
This slidename failed to match and is not a control E06-52_CG_1_UBIQ.ndpi
This slidename failed to match and is not a control E06-52_CG_1_Ubiq_plus.ndpi
This slidename failed to match and is not a control E06-52_CG_1_a-syn4B12.ndpi
This slidename failed to match and is not a control E06-52-CG-1-syn4B12.ndpi
This slidename failed to match and is not a control E06-52-CG-1-synB12.ndpi
This slidename failed to match and is not a control E06-52_CG_1_syn_4B12_minus.ndpi
This slidename failed to match and is not a control E06-52_CG_1_a-s

This slidename failed to match and is not a control 0589-44-AB_3.ndpi
This slidename failed to match and is not a control syn4B12bad.ndpi
This slidename failed to match and is not a control 0597-60-tau-minus.ndpi
This slidename failed to match and is not a control 0597-60-tau.ndpi
This slidename failed to match and is not a control A10-160-1S.ndpi
This slidename failed to match and is not a control backwards.ndpi
This slidename failed to match and is not a control E06-52_CG_1_Syn4B12.ndpi
This slidename failed to match and is not a control E04-76_F_Abeta.ndpi
This slidename failed to match and is not a control E06-52_CG_1_syn4B12_+.ndpi
This slidename failed to match and is not a control OS97-60_tau.ndpi
This slidename failed to match and is not a control OS97-60_tau_+.ndpi
This slidename failed to match and is not a control 1023337.svs
This slidename failed to match and is not a control 1023338.svs
This slidename failed to match and is not a control 1023339.svs
This slidename failed t

...

This slidename failed to match and is not a control BIEL.ndpi
This slidename failed to match and is not a control E07-48_T_pTDP-43.ndpi
This slidename failed to match and is not a control OS97-60_D_Ubiq.ndpi
This slidename failed to match and is not a control OS97-60_Hp_tau_4T8.ndpi
This slidename failed to match and is not a control OS97-60_T_syn4B12.ndpi
This slidename failed to match and is not a control 5-26-11_BIEL.ndpi
This slidename failed to match and is not a control LFB_CNTRL#2_for_A11-60.ndpi
This slidename failed to match and is not a control LFB_CNTRL#3_for_A11-60.ndpi
This slidename failed to match and is not a control LFB_CNTRL_for_A11-60.ndpi
This slidename failed to match and is not a control A11-60SC_R1.ndpi
This slidename failed to match and is not a control A11-62_SCd_7.ndpi
This slidename failed to match and is not a control E11-60SCdR1_GFAP_1to500.ndpi
This slidename failed to match and is not a control E11-60SCdR3_GFAP_1to500.ndpi
This slidename failed to match a

This slidename failed to match and is not a control E16-128_L2_GFAP.svs
This slidename failed to match and is not a control E16-128_L8_GFAP.svs
This slidename failed to match and is not a control E16-128_C2_GFAP.svs
This slidename failed to match and is not a control E16-128_C4_p62.svs
This slidename failed to match and is not a control E16-128_B1_pTDP.svs
This slidename failed to match and is not a control E16-128_C2_LFB.svs
This slidename failed to match and is not a control E16-128_L2_LFB.svs
This slidename failed to match and is not a control E16-128_C6_p62.svs
This slidename failed to match and is not a control E16-128_C8_LFB.svs
This slidename failed to match and is not a control E16-128_C2_p62.svs
This slidename failed to match and is not a control E16-128_L4_GFAP.svs
This slidename failed to match and is not a control E16-128_L6_LFB.svs
This slidename failed to match and is not a control E16-128_C8_pTDP.svs
This slidename failed to match and is not a control E16-128_A_LFB.svs
T

This slidename failed to match and is not a control 1006970.svs
This slidename failed to match and is not a control 1007160.svs
This slidename failed to match and is not a control 1012665.svs
This slidename failed to match and is not a control 1012694.svs
This slidename failed to match and is not a control 1016350.svs
This slidename failed to match and is not a control 1007397.svs
This slidename failed to match and is not a control 1007398.svs
This slidename failed to match and is not a control 1007399.svs
This slidename failed to match and is not a control 1007400.svs
This slidename failed to match and is not a control 1008446.svs
This slidename failed to match and is not a control E17-39_pTDP_R-2.svs
This slidename failed to match and is not a control E17-39_LFB_R-6.svs
This slidename failed to match and is not a control E17-39_GFAP_R7.svs
This slidename failed to match and is not a control E17-39_pTDP_R8.svs
This slidename failed to match and is not a control E17-39_LFB_R-9.svs
This

This slidename failed to match and is not a control OS170124_15_tau.svs
This slidename failed to match and is not a control OS170124_6_syn.svs
This slidename failed to match and is not a control OS170124_13_syn.svs
This slidename failed to match and is not a control OS170124_8_syn.svs
This slidename failed to match and is not a control OS170124_4_AB.svs
This slidename failed to match and is not a control OS170124_1_syn-001.svs
This slidename failed to match and is not a control OS170124_7_H&E.svs
This slidename failed to match and is not a control OS170124_12_AB.svs
This slidename failed to match and is not a control 1011685.svs
This slidename failed to match and is not a control 1011700.svs
This slidename failed to match and is not a control OS170518_10_H&E.svs
This slidename failed to match and is not a control OS170518_10_SYN.svs
This slidename failed to match and is not a control OS170518_10_TAU.svs
This slidename failed to match and is not a control OS170518_11_H&E.svs
This sliden

This slidename failed to match and is not a control E18-70_ 2 AB.svs
This slidename failed to match and is not a control E18-70_ 2 Biels.svs
This slidename failed to match and is not a control E18-70_ 1 AB.svs
This slidename failed to match and is not a control E18-70_ 8 Biels.svs
This slidename failed to match and is not a control E18-70_ 3 HE.svs
This slidename failed to match and is not a control E18-70_ 4 Biels.svs
This slidename failed to match and is not a control E18-70_ 8 HE.svs
This slidename failed to match and is not a control E18-70_ 5 HE.svs
This slidename failed to match and is not a control E18-70_ 12 HE.svs
This slidename failed to match and is not a control E18-70_ 13 HE.svs
This slidename failed to match and is not a control E18-70_ 14 HE.svs
This slidename failed to match and is not a control E18-70_ 1 HE.svs
This slidename failed to match and is not a control E18-70_ 7 HE.svs
This slidename failed to match and is not a control E18-70_ 16 HE.svs
This slidename failed

In [24]:
# hlprs.unmatchedFileNames

['ADRC36-04_LHp_pTDP_1to10k.ndpi',
 'ADRC36-04_F_pTDP-1to10K.ndpi',
 'ADRC36-04_RHp_pTDP_1to10k.ndpi',
 'E39-44_TAU.ndpi',
 'E39-44_BLANK.ndpi',
 'E39-44_AB.ndpi',
 'OS89-44_Ab.ndpi',
 'OS89-44_Ab_2.ndpi',
 'OS89-44_Ab_3.ndpi',
 'PSTVE_CNTRL_FOR_OS89-44_AB.ndpi',
 'E39-44_SYN.ndpi',
 '1028860.svs',
 '1028861.svs',
 '1028862.svs',
 '1028863.svs',
 '1028864.svs',
 '1028865.svs',
 '1028866.svs',
 '1028867.svs',
 '1028868.svs',
 '1028869.svs',
 '1028870.svs',
 '1028871.svs',
 '1028872.svs',
 '1028873.svs',
 '1028874.svs',
 '1028875.svs',
 '1028876.svs',
 '1028877.svs',
 '1028878.svs',
 '1028879.svs',
 '1028880.svs',
 '1028881.svs',
 '1028882.svs',
 '1028883.svs',
 '1028884.svs',
 '1028885.svs',
 '1028886.svs',
 '1028887.svs',
 '1028888.svs',
 '1028889.svs',
 '1028890.svs',
 '1028891.svs',
 '1028892.svs',
 '1028893.svs',
 '1028894.svs',
 '1028895.svs',
 '1028896.svs',
 '1028897.svs',
 '1028898.svs',
 '1028899.svs',
 '1028900.svs',
 '1028901.svs',
 '1028902.svs',
 '1028903.svs',
 '1028904.sv

In [8]:
## meta.region  meta.stain  meta.subject
basicSlideInfo = []
mdKeys = ['region','stain','subject','brain_region']

for subjyr in npSlideDict:
    for s in npSlideDict[subjyr]:
        slideData = {'folderId': subjyr,'slideName':s['name'],'itemId':s['_id']}
        
        for k in mdKeys:
            if k in s['meta']:
                slideData[k] = s['meta'][k]
        basicSlideInfo.append(slideData)
pd.DataFrame(basicSlideInfo).to_csv("adrcInitialMetadataExport.csv")

In [27]:
npByYearCase = {}
for k in npSlideDict:
    try:
        (year,caseId) = k.split("_")
        if year not in npByYearCase:
            npByYearCase[year]= []
            
        npByYearCase[year].append(caseId)
        
    except:
        print("Weird pattern for",k)

In [58]:
# from ipywidgets import Dropdown, interactive, HBox, interactive_output
# from collections import Counter
# npYearList = list(npByYearCase.keys())

# dpYear = Dropdown(options=npYearList,description='Year')
# dpCaseId = Dropdown(options=npByYearCase[npYearList[0]],description='CaseId')


# def on_change(*args):
#     value = dpYear.value
#     dpCaseId.options = npByYearCase[value]
# dpYear.observe(on_change, names='value')

# def analyzeNpCase(year, caseId):
#     print("Gutman needs a bath")
#     ### This will perform an analysis of a folder of images to see the number of slides 
#     ## And check the metadata
#     npKey = "%s_%s" % ( year,caseId)
#     slideList = npSlideDict[npKey]
    
#     ## Keys of interest
#     koi = ['region','regionId','subject','stain','Stain','Region']
#     slideKeys = {}
#     for k in koi:
#         slideKeys[k]= []
    
#     caseMeta = [] ## 
#     d=caseMeta ## Exposing this as a global
    
#     for s in slideList:
#         meta = s['meta']
#         for k in meta:
#             if k in koi:
#                 slideKeys[k].append(meta[k])
#         caseMeta.append( hlprs.validateSlide(s,caseId) )
                
                
#     ## Now go through and make things into counters so it's easier to see
#     for k in koi:
#         slideKeys[k] = Counter(slideKeys[k])
                
#     print("Unique RegionId Count: %d \t Distinct SubjID: \t%d"% (len(slideKeys['region']),len(slideKeys['subject'])))
#     print("Total slides found: %d" % len(slideList))  ## Will add in schema maps as well
#     df = pd.DataFrame(caseMeta)
# #     print(df)
#     if 'region' in df.columns:
#         print(df.region.value_counts())
#     else:
#         print("region tag not found..")
#         print([s['name'] for s in slideList])

    
# ui = HBox([dpYear,dpCaseId])
# w = interactive_output(analyzeNpCase, {'year':dpYear, 'caseId':dpCaseId})
# display(ui,w)

# ## Key I am interested in is npSchema going forward
# ## Need to figure out what to do with control slides..  will add a key to the Schema that's optional called controlSlide


Output()

In [9]:
# ## LEt's examine a specific case...  2020_E20-106
# import re

# caseId = '2020_E20-106'
# npSlideDict[caseId]

# adrcName_re = re.compile('(?P<subject>.\d\d-\d+)_(?P<blockID>\d+) (?P<stain>.*)\.[svs|ndpi]')

# ## Let me look at the filenames in this dictionary
# for slideName in [x['name'] for x in npSlideDict[caseId]]:
#     m =  adrcName_re.search(slideName)
#     if m:
#         print(slideName,m.groupdict())
#     elif ('CON' in slideName.upper() ):  ### This is probably a control slide
#         print("Control slide?",slideName)
#     else:
#         print("Shit nothing works",slideName)

E20-106_17 HE.svs {'subject': 'E20-106', 'blockID': '17', 'stain': 'HE'}
E20-106_5 AB.svs {'subject': 'E20-106', 'blockID': '5', 'stain': 'AB'}
E20-106_8 AB.svs {'subject': 'E20-106', 'blockID': '8', 'stain': 'AB'}
E20-106_11 HE.svs {'subject': 'E20-106', 'blockID': '11', 'stain': 'HE'}
E20-106_7 HE.svs {'subject': 'E20-106', 'blockID': '7', 'stain': 'HE'}
E20-106_10 AB.svs {'subject': 'E20-106', 'blockID': '10', 'stain': 'AB'}
E20-106_14 HE.svs {'subject': 'E20-106', 'blockID': '14', 'stain': 'HE'}
E20-106_9 Syn.svs {'subject': 'E20-106', 'blockID': '9', 'stain': 'Syn'}
E20-106_18 HE.svs {'subject': 'E20-106', 'blockID': '18', 'stain': 'HE'}
E20-106_11 Syn.svs {'subject': 'E20-106', 'blockID': '11', 'stain': 'Syn'}
E20-106_3 Syn.svs {'subject': 'E20-106', 'blockID': '3', 'stain': 'Syn'}
E20-106_8 Tau.svs {'subject': 'E20-106', 'blockID': '8', 'stain': 'Tau'}
E20-106_18 pTDP.svs {'subject': 'E20-106', 'blockID': '18', 'stain': 'pTDP'}
E20-106_2 Syn.svs {'subject': 'E20-106', 'blockID':